# Pre-processing
This notebook calculates building coverage as a percentage of each task, for every chosen project

## Global Parameters

In [ ]:
data_folder = "data/"

In [ ]:
in_drive = False  # True to mount a drive while working in Google Colab
if in_drive:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)

## Setup

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from datetime import datetime, timedelta
from tqdm.autonotebook import tqdm
from IPython.display import Markdown, display
!pip install utm
import utm

In [ ]:
def calculate_utm_zone(gdf: gpd.GeoDataFrame) -> str:
    """
    From a GeoDataFrame (GDF) in EPSG:4326, calculate an accurate enough UTM zone for it
    """
    assert gdf.crs == "EPSG:4326"

    bbox = gdf.geometry.total_bounds
    lat = bbox[3] - bbox[1]
    lon = bbox[2] - bbox[0]
    _, _, zone_number, _ = utm.from_latlon(lat, lon)
    if lat >= 0:
        epsg_number = "326" + str(zone_number).zfill(2)
    else:
        epsg_number = "327" + str(zone_number).zfill(2)

    return epsg_number


## Densities effect

In [ ]:
input_ids_filename = data_folder + "output_archived_projs_selected_ids.csv"
overwrite_intermediate_if_exists = False

display(Markdown("Reading selected project ids"))
input_data = pd.read_csv(input_ids_filename)

densities: pd.DataFrame = None

display(Markdown("CALCULATING BUILDING DENSITY FOR EVERY TASK OF EVERY PROJECT"))
for proj_id in tqdm(input_data['projectId'], unit="project(s)"):
    output_intermediate_filename = data_folder + "output_densities_" + str(proj_id) + ".geojson"
    output_intermediate_filename_exists = os.path.isfile(output_intermediate_filename)

    if output_intermediate_filename_exists and not overwrite_intermediate_if_exists:
        display(Markdown("[Project #" + str(proj_id) + "](https://tasks.hotosm.org/projects/" + str(proj_id) + ") already calculated"))
        grid_osm_difference = pd.read_csv(output_intermediate_filename)
    else:
        ## READ INPUT FILES ##
        display(Markdown("Reading input files for [project #" + str(proj_id) + "](https://tasks.hotosm.org/projects/" + str(proj_id) + ")"))

        # Read Project OSM building data
        input_osm_data_filename = data_folder + "output_proj_" + str(proj_id) + "_osm.geojson"
        osm_data = gpd.GeoDataFrame.from_file(input_osm_data_filename)

        # Read Project Grid
        input_proj_grid_filename = data_folder + "output_proj_" + str(proj_id) + "_grid.geojson"
        proj_grid = gpd.GeoDataFrame.from_file(input_proj_grid_filename)

        ## DATA TRANSFORMATION AND ANALYSIS ##
        # Fix mixed types in OSM data
        osm_data = osm_data.loc[osm_data.geometry.geometry.type=='MultiPolygon']
        # OSM multipolygons get treated as multilinestrings by Bunting Labs API
        # We ignore those due to complexity of converting them to proper multipolygons without outer/inner info

        # Reproject to UTM - Necessary for area calculation
        epsg_number = calculate_utm_zone(proj_grid)
        display(Markdown("Reprojecting to UTM, EPSG:" + epsg_number))
        proj_grid.to_crs("EPSG:" + epsg_number, inplace=True)
        osm_data.to_crs("EPSG:" + epsg_number, inplace=True)

        # Calculate task grid cell area
        display(Markdown("Calculating grid cell area"))
        proj_grid["area_sqm"] = proj_grid.geometry.area

        # Delete obvious errors in OSM building data
        # (buildings with area greater than 100km2)
        osm_data.drop(osm_data.loc[osm_data.area > 1e+08].index, inplace=True)  # 1e+08 m2 = 100 km2

        # Overlay OSM building data onto the grid, to get the area not covered by them
        display(Markdown("Calculating building area"))
        grid_osm_difference = gpd.overlay(proj_grid, osm_data, how="difference")
        # https://geopandas.org/en/stable/docs/reference/api/geopandas.overlay.html
        # See: https://geopandas.org/en/stable/docs/user_guide/set_operations.html

        # Calculate areas not covered by buildings
        grid_osm_difference["area_not_building"] = grid_osm_difference.area
        # This assertion fails if any given building covers a whole grid cell
        # Delete the assertion if such case happens in a logical way for a given project
        # The assertion is here to check for obvious building data errors
        ## Fails project 12101 task 143 https://tasks.hotosm.org/projects/12101/tasks?search=143&page=1
        #assert len(grid_osm_difference) == len(proj_grid)

        # Calculate area covered by buildings
        grid_osm_difference["area_with_building"] = grid_osm_difference["area_sqm"] - grid_osm_difference["area_not_building"]

        # Calculate percentage covered by building
        display(Markdown("Calculating building density"))
        grid_osm_difference["percentage_area_covered_by_building"] = (grid_osm_difference["area_with_building"] / grid_osm_difference["area_sqm"]) * 100.0

        # Select relevant columns
        grid_osm_difference = grid_osm_difference[["taskId", "area_sqm", "percentage_area_covered_by_building"]]
        grid_osm_difference["projId"] = str(proj_id)

        # Save intermediate status to recover from an eventual failure
        display(Markdown("Saving intermediate output"))
        grid_osm_difference.to_csv(output_intermediate_filename, index=False)

    # Append to densities DataFrame
    if densities is None:
        densities = grid_osm_difference.copy()
    else:
        densities = pd.concat([densities, grid_osm_difference.copy()],
                              ignore_index=True)

    display(Markdown("Project #" + str(proj_id) + " done!"))

output_filename = data_folder + "output_densities.csv"
display(Markdown("Saving output"))
densities.to_csv(output_filename, index=False)
display(Markdown("Output saved to " + output_filename))

display(Markdown("Finished!"))